In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, GlobalMaxPooling1D

# Load the dataset
file_path = '/content/dataset.tsv'
data = pd.read_csv(file_path, delimiter='\t')

# Constants
max_len = 100  # Maximum length of text sequences
max_words = 10000  # Maximum number of words to consider in the vocabulary

# Split the data into features and labels
X = data['text'].values
y_binary = data['label'].values  # Binary classification labels
y_category = data['Pattern Category'].values  # Multi-class classification labels

# Tokenize the text
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len)

# Encode the multi-class labels
label_encoder = LabelEncoder()
y_category_encoded = label_encoder.fit_transform(y_category)

# Split the data
X_train, X_test, y_binary_train, y_binary_test, y_category_train, y_category_test = train_test_split(X_pad, y_binary, y_category_encoded, test_size=0.2)

# Model Creation
input_text = Input(shape=(max_len,))
embedding = Embedding(max_words, 50)(input_text)
lstm = LSTM(64, return_sequences=True)(embedding)
global_max_pool = GlobalMaxPooling1D()(lstm)
binary_pred = Dense(1, activation='sigmoid', name='binary_output')(global_max_pool)  # Binary classification output
category_dense = Dense(64, activation='relu')(global_max_pool)
category_pred = Dense(len(label_encoder.classes_), activation='softmax', name='category_output')(category_dense)  # Multi-class classification output

model = Model(inputs=input_text, outputs=[binary_pred, category_pred])

# Compile the model
model.compile(optimizer='adam',
              loss={'binary_output': 'binary_crossentropy', 'category_output': 'sparse_categorical_crossentropy'},
              metrics=['accuracy'])

# Model Training
model.fit(X_train, {'binary_output': y_binary_train, 'category_output': y_category_train}, epochs=15, validation_split=0.2)

# Evaluation
model.evaluate(X_test, {'binary_output': y_binary_test, 'category_output': y_category_test})


Epoch 1/15
48/48 [==============================] - 8s 91ms/step - loss: 2.3170 - binary_output_loss: 0.6951 - category_output_loss: 1.6219 - binary_output_accuracy: 0.4957 - category_output_accuracy: 0.4937 - val_loss: 2.1812 - val_binary_output_loss: 0.6916 - val_category_output_loss: 1.4896 - val_binary_output_accuracy: 0.5066 - val_category_output_accuracy: 0.4934
Epoch 2/15
48/48 [==============================] - 3s 66ms/step - loss: 2.0805 - binary_output_loss: 0.6854 - category_output_loss: 1.3951 - binary_output_accuracy: 0.5362 - category_output_accuracy: 0.5030 - val_loss: 2.1579 - val_binary_output_loss: 0.6751 - val_category_output_loss: 1.4828 - val_binary_output_accuracy: 0.8011 - val_category_output_accuracy: 0.4934
Epoch 3/15
48/48 [==============================] - 3s 66ms/step - loss: 1.9489 - binary_output_loss: 0.6322 - category_output_loss: 1.3167 - binary_output_accuracy: 0.8275 - category_output_accuracy: 0.5030 - val_loss: 1.8669 - val_binary_output_loss: 0.564

[0.7564239501953125,
 0.21794772148132324,
 0.5384761691093445,
 0.9216101765632629,
 0.8622881174087524]

In [ ]:
model.save('Dark_Pattern_buster.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
